In [1]:
from sentence_transformers import SentenceTransformer, util
import json
from tqdm import tqdm
import numpy as np
import evaluate
from nltk.tokenize import word_tokenize
import random
import nltk
from scipy.spatial.distance import jensenshannon
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import jensenshannon

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

2024-07-18 00:41:42.669929: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 00:41:42.857806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 00:41:42.857851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 00:41:42.859479: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 00:41:42.891722: I tensorflow/core/platform/cpu_feature_guar

In [6]:
"Agreeableness - high, Neuroticism - high\nin the a.m.this has toassistant\n".split("\n")[1][:-9]

'in the a.m.this has to'

In [2]:
file_path = "/home/jiaruil5/personality/llm_personality/llm_bigfive/generator/outputs/w_tokens_1000/generated_predictions.jsonl"
ground_true = []
prediction = []
for line in open(file_path, 'r').readlines():
    line = json.loads(line)
    prompt = line['prompt'].split(" Neuroticism - ")[1]
    if prompt[:4] == 'high':
        prompt = prompt[4:]
    elif prompt[:3] == 'low':
        prompt = prompt[3:]
    else:
        prompt = prompt[6:]
    prompt = prompt.split("\n")[1][:-9]
    ground_true.append(prompt + " " + line['label'])
    prediction.append(prompt + " " + line['predict'])

def compute_token_overlap_ratio(text1, text2):
    tokens1 = set(word_tokenize(text1.lower()))
    tokens2 = set(word_tokenize(text2.lower()))
    overlap = len(tokens1.intersection(tokens2))
    ratio = overlap / max(len(tokens1), len(tokens2))
    return ratio

def compute_jsd(sentence1, sentence2):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    sentences = [sentence1, sentence2]
    embeddings = model.encode(sentences)
    def normalize_vector(vector):
        vector = np.abs(vector)
        return vector / np.sum(vector)
    norm_embeddings = np.array([normalize_vector(embedding) for embedding in embeddings])
    jsd = jensenshannon(norm_embeddings[0], norm_embeddings[1], base=2)
    similarity = 1 - jsd
    return similarity

In [3]:
print(ground_true[0])

afternoon beer and football. love bank hol weekends! just wish the san miguel was cold!


In [4]:
print(prediction[0])

afternoon beer and football. love it


In [5]:
#Compute embedding to gain similarity
similarity_list = []
for i in tqdm(range(len(prediction)), desc = "Proccessing Cosine Similarity: "):
    embedding_1= model.encode(ground_true[i], convert_to_tensor=True)
    embedding_2 = model.encode(prediction[i], convert_to_tensor=True)
    similarity_list.append(util.pytorch_cos_sim(embedding_1, embedding_2).cpu().detach().numpy())
print(f"Average Cosine Sentence Similarity: {np.mean(similarity_list)}")

overlap_ratios = []
for i in tqdm(range(len(prediction)), desc = "Proccessing Token Overlapping: "):
    true_text = ground_true[i]
    pred_text = prediction[i]
    
    overlap_ratio = compute_token_overlap_ratio(true_text, pred_text)
    overlap_ratios.append(overlap_ratio)
average_overlap_ratio = np.mean(overlap_ratios)
print(f"Average Token Overlapping Ratio: {average_overlap_ratio}")

jsd_list = []
for i in tqdm(range(len(prediction)), desc = "Proccessing JSD Similarity: "):
    true_text = ground_true[i]
    pred_text = prediction[i]
    
    jsd_score = compute_jsd(true_text, pred_text)
    jsd_list.append(jsd_score)

average_jsd_score = np.mean(jsd_list)
print(f"Average JSD Sentence Similarity: {average_jsd_score}")

Proccessing Cosine Similarity: 100%|██████████| 1000/1000 [01:13<00:00, 13.63it/s]


Average Cosine Sentence Similarity: 0.5262521505355835


Proccessing Token Overlapping: 100%|██████████| 1000/1000 [00:00<00:00, 1738.92it/s]


Average Token Overlapping Ratio: 0.3722497367190912


Proccessing JSD Similarity: 100%|██████████| 1000/1000 [12:32<00:00,  1.33it/s]

Average JSD Sentence Similarity: 0.6180709342678702


In [6]:
cosine_similarity = np.mean(similarity_list)
print("Average Cosine Sentence Similarity: {:.4f}".format(cosine_similarity))
print("Average Token Overlapping Ratio: {:.4f}".format(average_overlap_ratio))
print("Average JSD Sentence Similarity: {:.4f}".format(average_jsd_score))

Average Cosine Sentence Similarity: 0.5263
Average Token Overlapping Ratio: 0.3722
Average JSD Sentence Similarity: 0.6181


# Test Evaluation

In [ ]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import jensenshannon
import numpy as np

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

sentences = ["This is the first sentence.", "This is the third sentence."]

embeddings = model.encode(sentences)

def normalize_vector(vector):
    vector = np.abs(vector)
    return vector / np.sum(vector)

norm_embeddings = np.array([normalize_vector(embedding) for embedding in embeddings])

jsd = jensenshannon(norm_embeddings[0], norm_embeddings[1], base=2)


similarity = 1 - jsd
return similarity

# Personality Alpaca Format